In [5]:
import cv2  
import numpy as np
import sys,os
import tkinter as tk
from tkinter import ttk
from PIL import Image, ImageTk
  
# Load Yolo  
print("LOADING YOLO")  
net = cv2.dnn.readNet("C:\\Users\\obvious\\Desktop\\sight\\yolov3.weights", "C:\\Users\\obvious\\Desktop\\sight\\yolov3.cfg")  
# Save all the names in file o the list classes  
classes = []  
with open("C:\\Users\\obvious\\Desktop\\sight\\coco.names", "r") as f:  
    classes = [line.strip() for line in f.readlines()]  
# Get layers of the network  
layer_names = net.getLayerNames()  
# Determine the output layer names from the YOLO model  
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]  
print("YOLO LOADED")

LOADING YOLO
YOLO LOADED


In [22]:
video_capture = cv2.VideoCapture("C:\\Users\\obvious\\Desktop\\test_git\\goturn_tracker\\16run.mp4")

while True:
    # Capture frame-by-frame
    re,img = video_capture.read()
    img = cv2.resize(img, None, fx=0.4, fy=0.4)
    height, width, channels = img.shape
 
    # USing blob function of opencv to preprocess image
    blob = cv2.dnn.blobFromImage(img, 1 / 255.0, (416, 416),
     swapRB=True, crop=False)
    #Detecting objects
    net.setInput(blob)
    outs = net.forward(output_layers)
 
    # Showing informations on the screen
    class_ids = []
    confidences = []
    boxes = []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                # Object detected
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
 
                # Rectangle coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
 
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)
    
    #We use NMS function in opencv to perform Non-maximum Suppression
    #we give it score threshold and nms threshold as arguments.
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    font = cv2.FONT_HERSHEY_PLAIN
    colors = np.random.uniform(0, 255, size=(len(classes), 3))
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            color = colors[class_ids[i]]
            cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
            cv2.putText(img, label, (x, y + 30), font, 2, color, 3)
 
    cv2.imshow("Image",cv2.resize(img, (800,600)))
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
video_capture.release()
cv2.destroyAllWindows()

In [23]:
video_capture = cv2.VideoCapture("C:\\Users\\obvious\\Desktop\\test_git\\goturn_tracker\\16run.mp4")
def mouse_callback(event, x, y, flags, param):
    global selected_box_index
    
    if event == cv2.EVENT_LBUTTONDOWN:
        selected_box_index = None
        for idx, (bbox, color) in enumerate(zip(boxes, colors)):
            x1, y1, x2, y2 = bbox
            if x1 <= x <= x2 and y1 <= y <= y2:
                selected_box_index = idx
                print(f"选中了 bounding box {selected_box_index + 1}")
                break
selected_box_index = None
font = cv2.FONT_HERSHEY_PLAIN


# 创建 OpenCV 窗口并设置鼠标回调函数
cv2.namedWindow("Image")
cv2.setMouseCallback("Image", mouse_callback)


while True:
    # Capture frame-by-frame
    ret, img = video_capture.read()
    if not ret:
        break

    img = cv2.resize(img, None, fx=0.4, fy=0.4)
    height, width, channels = img.shape

    # Using blob function of opencv to preprocess image
    blob = cv2.dnn.blobFromImage(img, 1 / 255.0, (416, 416), swapRB=True, crop=False)

    # Detecting objects
    net.setInput(blob)
    outs = net.forward(output_layers)

    # Showing informations on the screen
    class_ids = []
    confidences = []
    boxes = []
    colors = []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                # Object detected
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                # Rectangle coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                 # Check if class_id is within bounds
                if class_id >= len(classes):
                    class_id = len(classes) - 1

                # Append bounding box and color information
                boxes.append((x, y, x+w, y+h))
                colors.append(np.random.randint(0, 255, size=(3,)).tolist())
                confidences.append(float(confidence))
                class_ids.append(class_id)

# 获取 NMS 的索引
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    selected_box_index_drawn = False
    # 存储每个类别目标的最高置信度及其对应的框坐标
    highest_confidences = {}
    for idx, (bbox, class_id) in enumerate(zip(boxes, class_ids)):
        if class_id not in highest_confidences or confidences[idx] > highest_confidences[class_id][0]:
            highest_confidences[class_id] = (confidences[idx], bbox)

# 在图像上绘制 bounding boxes 和选中状态
    for class_id, (highest_confidence, bbox) in highest_confidences.items():
        x1, y1, x2, y2 = bbox
        label = str(classes[class_id])
        color = colors[class_id]
        cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)
        cv2.putText(img, label, (x1, y1 - 10), font, 1, color, 2)
        if selected_box_index is not None and selected_box_index == class_id:
            color = (255, 0, 0)  # 设置选定 bounding box 的颜色为红色
            cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)
            center_x = (x1 + x2) // 2
            center_y = (y1 + y2) // 2
            cv2.drawMarker(img, (center_x, center_y), color, cv2.MARKER_CROSS, 20, 2)

    selected_box_index_drawn = True  # 设置标志为 True

    cv2.imshow("Image", cv2.resize(img, (800, 600)))
    key = cv2.waitKey(1)
    if key == ord('q'):
        break
video_capture.release()
cv2.destroyAllWindows()

选中了 bounding box 1
选中了 bounding box 1


In [ ]:
for idx, (bbox, color) in enumerate(zip(boxes, colors)):
        x1, y1, x2, y2 = bbox
        label = str(classes[class_ids[idx]])
        cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)
        cv2.putText(img, label, (x1, y1 - 10), font, 1, color, 2)

    # 绘制选定框的特殊样式
    if selected_box_index is not None:
        bbox = boxes[selected_box_index]
        x1, y1, x2, y2 = bbox
        color = (255, 0, 0)  # 设置选定 bounding box 的颜色为红色
        cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)
        center_x = (x1 + x2) // 2
        center_y = (y1 + y2) // 2
        cv2.drawMarker(img, (center_x, center_y), color, cv2.MARKER_CROSS, 20, 2)

In [ ]:
    for idx, (bbox, color) in enumerate(zip(boxes, colors)):
        x1, y1, x2, y2 = bbox
        if idx == selected_box_index:
            color = (255, 0, 0)  # 设置选定 bounding box 的颜色为红色
            # 在选定 bounding box 上绘制一个矩形框，以标示为选定状态
            cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)
            # 获取选定 bounding box 的中心坐标
            center_x = (x1 + x2) // 2
            center_y = (y1 + y2) // 2
            # 在选定 bounding box 的中心位置绘制一个十字标记
            cv2.drawMarker(img, (center_x, center_y), color, cv2.MARKER_CROSS, 20, 2)
            selected_box_index_drawn = True
        elif idx in indexes:
            x, y, w, h = bbox
            label = str(classes[class_ids[idx]])
            color = colors[class_ids[idx]]
            cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
            cv2.putText(img, label, (x, y + 30), font, 2, color, 3)